In [1]:
import sys
sys.path.insert(0, '/Users/waihengsoh/Desktop/Simply Solution/vehicle_dis/Equities_Price_Prediction_XAI')

In [2]:
!pip install -r ../requirements.txt


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification, InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# news = historical_news.get_all_news(date(2021, 8, 1), date(2021, 10, 31))
train = pd.read_csv('/Users/waihengsoh/Desktop/Simply Solution/vehicle_dis/Equities_Price_Prediction_XAI/Functions/Historical_News/news/train.csv')
train = train.loc[:, ['content', 'price_change']]
X_train = train['content'].tolist()
y_train = train['price_change'].tolist()

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

test = pd.read_csv('/Users/waihengsoh/Desktop/Simply Solution/vehicle_dis/Equities_Price_Prediction_XAI/Functions/Historical_News/news/test.csv')
test = test.loc[:, ['content', 'price_change']]
X_test = test['content'].tolist()
y_test = test['price_change'].tolist()

print(len(X_train), len(y_train), len(X_val), len(y_val), len(X_test), len(y_test))


In [ ]:
from Functions.Historical_News import historical_news
from datetime import date, timedelta
news = historical_news.get_all_news(date(2021, 8, 1), date(2021, 10, 31))
news = news.loc[:, ['content', 'price_change']]


extracting news for AAPL.US ...
extracting news for MSFT.US ...
extracting news for AMZN.US ...
extracting news for TSLA.US ...
extracting news for GOOGL.US ...
extracting news for GOOG.US ...


In [ ]:
news

In [ ]:
train.head()

In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None,
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None,
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True,
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train[:47000], train[47000:], 'content', 'price_change')

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, epsilon=1e-07, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=3, steps_per_epoch=100, validation_data=validation_data)

In [ ]:
tf_batch = tokenizer(X_test[:100], padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
correct, wrong = 0, 0
for i in range(len(X_test[:100])):
  print("predicted:", labels[label[i]], "actual:", labels[int(y_test[i])])
  if labels[label[i]] == labels[int(y_test[i])]:
    correct += 1
  else:
    wrong += 1
print(correct / (correct + wrong))

references from : https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671